In [37]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tqdm import tqdm
import sys



In [26]:
# Load metadata
df_metadata = pd.read_csv("../data/segmentation/sample_archive/metadata.csv")

# Split df_metadata into train and test by the "split" column
train_df = df_metadata.loc[df_metadata['split'] == 'train']
test_df = df_metadata.loc[df_metadata['split'] == 'test']

train_df

,image_id,split,sat_image_path,mask_path
0,992507,train,train/992507_sat.jpg,train/992507_mask.png
1,994520,train,train/994520_sat.jpg,train/994520_mask.png
2,995492,train,train/995492_sat.jpg,train/995492_mask.png
3,997521,train,train/997521_sat.jpg,train/997521_mask.png
4,998002,train,train/998002_sat.jpg,train/998002_mask.png


In [27]:
# Load classes
df_classes = pd.read_csv("../data/segmentation/sample_archive/class_dict.csv")
df_classes

,name,r,g,b
0,urban_land,0,255,255
1,agriculture_land,255,255,0
2,rangeland,255,0,255
3,forest_land,0,255,0
4,water,0,0,255
5,barren_land,255,255,255
6,unknown,0,0,0


In [28]:
# Preprocess data
def preprocess_data(metadata):
    X = []
    y = []

    for index, row in metadata.iterrows():
        image_path = "../data/segmentation/sample_archive/" + row['sat_image_path']
        mask_path = "../data/segmentation/sample_archive/" +  row['mask_path']

        # Load image
        image = cv2.imread(image_path)

        # Resize image if needed
        image = cv2.resize(image, (128, 128))

        # Load mask
        mask = cv2.imread(mask_path)#, cv2.IMREAD_GRAYSCALE
        mask = cv2.resize(mask, (128, 128))

        X.append(image)
        y.append(mask)

    return np.array(X), np.array(y)

# Prepare dataset for training
X, y = preprocess_data(train_df)

In [29]:
# Split train dataset into further labelled train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(4, 128, 128, 3) (1, 128, 128, 3) (4, 128, 128, 3) (1, 128, 128, 3)


In [30]:
# Define RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)

In [31]:
# Flatten the images and masks
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_test_flat = y_test.reshape(y_test.shape[0], -1)


In [32]:
# Train the model
model.fit(X_train_flat, y_train_flat)

# Make predictions
y_pred_flat = model.predict(X_test_flat)

In [38]:
# Save the current threshold for array printing
np.set_printoptions(threshold=sys.maxsize)


In [40]:
y_pred_flat

array([[  0, 255, 255,   0, 255, 255,   0, 255, 255,   0, 255, 255,   0,
        255, 255,   0, 255, 255,   0, 255, 255,   0, 255,   0,   0, 255,
          0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,
          0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0,
        255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,
          0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,
          0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0,
        255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,
          0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,
          0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0,
        255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,
          0,   0, 255,   0,   0, 255,   0,   0, 255,   0,   0, 255,   0,
          0, 255,   0, 255, 255, 255, 255, 255, 255, 255,   0, 255, 255,
          0, 255, 255,   0, 255, 255,   0, 255, 255

In [33]:
# Evaluate the model
accuracy = accuracy_score(y_test_flat, y_pred_flat)
print("Accuracy:", accuracy)

ValueError: multiclass-multioutput is not supported

In [ ]:
# Reshape predictions back to original shape
y_pred = y_pred_flat.reshape(y_test.shape)

# Convert predictions to original class labels
def convert_to_class(predictions):
    classes = []
    for image in predictions:
        img_classes = []
        for row in image:
            row_classes = []
            for pixel in row:
                class_index = np.argmax(pixel)
                class_name = df_classes.iloc[class_index]['name']
                row_classes.append(class_name)
            img_classes.append(row_classes)
        classes.append(img_classes)
    return np.array(classes)

y_pred_classes = convert_to_class(y_pred)

# Classification report
print(classification_report(y_test, y_pred_classes))